In [84]:
import pandas as pd
import numpy as np
from scipy.stats import mode

# Genero y exploro el Dataframe de los vuelos con sus partes meteorológicos METAR


In [2]:
df1 = pd.read_csv("../data/flights/0.part")
df2 = pd.read_csv("../data/flights/1.part")

In [3]:
mt = pd.read_csv("../data/metars/metars_2017_2023.csv")

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
fl = pd.concat([df1,df2])

In [6]:
fl.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 922599 entries, 0 to 424612
Data columns (total 31 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Date_time          922599 non-null  object 
 1   flight_id          922599 non-null  int64  
 2   cod_flight_IATA    922599 non-null  object 
 3   cod_flight_ICAO    922599 non-null  object 
 4   Date               922599 non-null  object 
 5   week_day           922599 non-null  object 
 6   status             922599 non-null  object 
 7   airliner           922599 non-null  object 
 8   cod_airliner_IATA  922599 non-null  object 
 9   cod_airliner_ICAO  922599 non-null  object 
 10  Scheduled_dep      922599 non-null  object 
 11  depart_time        922599 non-null  object 
 12  dep_situation      922599 non-null  object 
 13  dep_mins_of_delay  922599 non-null  object 
 14  city               922599 non-null  object 
 15  cod_airport_IATA   922599 non-null  object 
 16  co

# Me quedo solo con los vuelos que tienen registrados el número de minutos de retraso

In [7]:
fl = fl[(fl.arr_mins_of_delay != "-")]

In [8]:
fl.status.value_counts()

Landed    890376
Name: status, dtype: int64

# Elimino columnas con información redundante o sin utilidad para la predicción

In [9]:
fl[fl.cod_flight_IATA == "-"].shape

(0, 31)

In [10]:
fl[fl.cod_airliner_ICAO == "-"].shape

(0, 31)

In [11]:
fl[fl.cod_airport_ICAO == "-"].shape

(0, 31)

In [12]:
fl.head()

,Date_time,flight_id,cod_flight_IATA,cod_flight_ICAO,Date,week_day,status,airliner,cod_airliner_IATA,cod_airliner_ICAO,Scheduled_dep,depart_time,dep_situation,dep_mins_of_delay,city,cod_airport_IATA,cod_airport_ICAO,arrival,arr_situation,arr_mins_of_delay,duration,resta,Metar_id,Day,Hour,Condition,Temperature,Wind,Gusts,Relative_hum,Pressure
0,2023-10-31 23:59:00,1,IB6833,IBE6833,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,23:59,00:34,late,35,Santiago,SCL,SCEL,09:13 UTC-03,early,-6,13h,41,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0
1,2023-10-31 23:59:00,3,IB6011,IBE6011,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,23:59,00:24,late,25,Montevideo,MVD,SUMU,08:48 UTC-03,early,-6,12h,31,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0
2,2023-10-31 23:59:00,2,IB6841,IBE6841,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,23:59,00:32,late,33,Buenos Aires,EZE,SAEZ,08:29 UTC-03,on time,0,12h,33,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0
3,2023-10-31 23:55:00,6,IB6827,IBE6827,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,23:55,00:27,late,32,Sao Paulo,GRU,SBGR,06:49 UTC-03,early,-15,10h,47,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0
4,2023-10-31 23:55:00,4,IB6589,IBE6589,2023-10-31,Tuesday,Landed,Iberia,IB,IBE,23:55,00:19,late,24,Bogota,BOG,SKBO,03:52 UTC-05,early,-27,10h,51,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0


In [13]:
columnas_irr = ['cod_airliner_ICAO','cod_airport_ICAO','cod_flight_ICAO','Date','flight_id','status','airliner','Scheduled_dep','depart_time',
                'dep_situation','dep_mins_of_delay','arrival','arr_situation']
fl = fl.drop(columns=columnas_irr)

In [14]:
fl.head()

,Date_time,cod_flight_IATA,week_day,cod_airliner_IATA,city,cod_airport_IATA,arr_mins_of_delay,duration,resta,Metar_id,Day,Hour,Condition,Temperature,Wind,Gusts,Relative_hum,Pressure
0,2023-10-31 23:59:00,IB6833,Tuesday,IB,Santiago,SCL,-6,13h,41,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0
1,2023-10-31 23:59:00,IB6011,Tuesday,IB,Montevideo,MVD,-6,12h,31,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0
2,2023-10-31 23:59:00,IB6841,Tuesday,IB,Buenos Aires,EZE,0,12h,33,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0
3,2023-10-31 23:55:00,IB6827,Tuesday,IB,Sao Paulo,GRU,-15,10h,47,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0
4,2023-10-31 23:55:00,IB6589,Tuesday,IB,Bogota,BOG,-27,10h,51,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0


In [15]:
fl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 890376 entries, 0 to 424612
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Date_time          890376 non-null  object 
 1   cod_flight_IATA    890376 non-null  object 
 2   week_day           890376 non-null  object 
 3   cod_airliner_IATA  890376 non-null  object 
 4   city               890376 non-null  object 
 5   cod_airport_IATA   890376 non-null  object 
 6   arr_mins_of_delay  890376 non-null  object 
 7   duration           890376 non-null  object 
 8   resta              890376 non-null  int64  
 9   Metar_id           890376 non-null  int64  
 10  Day                890376 non-null  object 
 11  Hour               890376 non-null  object 
 12  Condition          890376 non-null  object 
 13  Temperature        890376 non-null  int64  
 14  Wind               890376 non-null  int64  
 15  Gusts              890376 non-null  int64  
 16  Re

# Genero una columna con el número de la semana en el año para recoger la estacionalidad del vuelo

In [16]:
# Convertir la columna 'Day' al tipo de dato datetime
fl['Day'] = pd.to_datetime(fl['Day'])

# Extraer el número de semana y el año de cada fecha
fl['NumSemana'] = fl['Day'].dt.strftime('%U')  # '%U' devuelve el número de semana del año (de 00 a 53)
# fl['Año'] = fl['Day'].dt.year


# Elimino Outliers de la columna objetivo

In [17]:
fl['arr_mins_of_delay'] = fl['arr_mins_of_delay'].astype(int)

In [18]:
fl = fl[fl.arr_mins_of_delay <= 360]

In [19]:
fl = fl[fl.arr_mins_of_delay >= -60]

In [20]:
fl[fl.arr_mins_of_delay > 120].shape

(7486, 19)

# Normalizo los valores de la columna 'duration' y elimino Outliers 

In [22]:
fl = fl[fl.duration != "23h"]
fl = fl[fl.duration != "14h"]
fl = fl[fl.duration != "15h"]
fl = fl[fl.duration != "16h"]
fl = fl[fl.duration != "30m"]

In [23]:
# Definir una función para la conversión
def convertir_a_minutos(valor):
    if pd.isna(valor) or valor == "-":
        return valor  # Dejar como está si es NaN o "-"
    
    if 'h' in valor:
        return valor  # Dejar como está si ya está en formato de horas
    
    if 'm' in valor:
        total_minutos = int(valor.split('m')[0])
    else:
        return valor  # Dejar como está si no tiene ni 'h' ni 'm'

    # Convertir a 1h si los minutos son mayores a 30, a 30m si son menores o iguales a 30
    if total_minutos > 30:
        return '1h'
    else:
        return '30m'



In [24]:
# Aplicar la función a la columna 'duration'
fl['duration'] = fl['duration'].apply(convertir_a_minutos)


In [26]:
fl.shape

(880762, 19)

# Reemplazo los valores nulos en la columna 'duration' con la moda de los vuelos con el mismo código

In [27]:
# Reemplazar los valores '-' con NaN en 'duration'
fl['duration'] = fl['duration'].replace('-', pd.NA)

# Calcular la moda dentro de cada grupo definido por 'cod_flight_IATA'
fl['duration'] = fl.groupby('cod_flight_IATA')['duration'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA)


In [28]:
fl.groupby('duration').agg({'arr_mins_of_delay':"mean"}).sort_values(by = 'arr_mins_of_delay', ascending = False)

,arr_mins_of_delay
duration,
5h,6.597011
4h,6.036531
10h,2.780310
6h,1.533333
1h,1.215314
3h,0.671978
2h,-0.537649
9h,-0.850367
8h,-2.371693


In [29]:
fl = fl.sort_values(by='Date_time', ascending=False)
fl = fl.reset_index(drop=True)
fl.head()

,Date_time,cod_flight_IATA,week_day,cod_airliner_IATA,city,cod_airport_IATA,arr_mins_of_delay,duration,resta,Metar_id,Day,Hour,Condition,Temperature,Wind,Gusts,Relative_hum,Pressure,NumSemana
0,2023-10-31 23:59:00,IB6833,Tuesday,IB,Santiago,SCL,-6,13h,41,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0,44
1,2023-10-31 23:59:00,IB6841,Tuesday,IB,Buenos Aires,EZE,0,12h,33,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0,44
2,2023-10-31 23:59:00,IB6011,Tuesday,IB,Montevideo,MVD,-6,12h,31,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0,44
3,2023-10-31 23:55:00,IB6827,Tuesday,IB,Sao Paulo,GRU,-15,10h,47,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0,44
4,2023-10-31 23:55:00,UX41,Tuesday,UX,Buenos Aires,EZE,-35,12h,51,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0,44


# Convierto la columna duration a numérica

In [30]:
fl['duration'] = pd.to_numeric(fl['duration'].str.replace('h', ''), errors='coerce')

In [31]:
fl.head()

,Date_time,cod_flight_IATA,week_day,cod_airliner_IATA,city,cod_airport_IATA,arr_mins_of_delay,duration,resta,Metar_id,Day,Hour,Condition,Temperature,Wind,Gusts,Relative_hum,Pressure,NumSemana
0,2023-10-31 23:59:00,IB6833,Tuesday,IB,Santiago,SCL,-6,13.0,41,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0,44
1,2023-10-31 23:59:00,IB6841,Tuesday,IB,Buenos Aires,EZE,0,12.0,33,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0,44
2,2023-10-31 23:59:00,IB6011,Tuesday,IB,Montevideo,MVD,-6,12.0,31,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0,44
3,2023-10-31 23:55:00,IB6827,Tuesday,IB,Sao Paulo,GRU,-15,10.0,47,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0,44
4,2023-10-31 23:55:00,UX41,Tuesday,UX,Buenos Aires,EZE,-35,12.0,51,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0,44


# Genero una columna con el momento del dia del vuelo, para recoger la variabilidad en los retrasos según el momento del día

In [32]:
fl['Date_time'] = pd.to_datetime(fl['Date_time'])
fl['momento_del_dia'] = pd.cut(fl['Date_time'].dt.hour,
                               bins=[0, 6, 12, 18, 24],
                               labels=['Early Morning', 'Morning', 'Afternoon', 'Night'],
                               right=False)


In [33]:
fl.head()

,Date_time,cod_flight_IATA,week_day,cod_airliner_IATA,city,cod_airport_IATA,arr_mins_of_delay,duration,resta,Metar_id,Day,Hour,Condition,Temperature,Wind,Gusts,Relative_hum,Pressure,NumSemana,momento_del_dia
0,2023-10-31 23:59:00,IB6833,Tuesday,IB,Santiago,SCL,-6,13.0,41,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0,44,Night
1,2023-10-31 23:59:00,IB6841,Tuesday,IB,Buenos Aires,EZE,0,12.0,33,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0,44,Night
2,2023-10-31 23:59:00,IB6011,Tuesday,IB,Montevideo,MVD,-6,12.0,31,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0,44,Night
3,2023-10-31 23:55:00,IB6827,Tuesday,IB,Sao Paulo,GRU,-15,10.0,47,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0,44,Night
4,2023-10-31 23:55:00,UX41,Tuesday,UX,Buenos Aires,EZE,-35,12.0,51,1,2023-10-31,23:30,Fair,8,3,0,93,1017.0,44,Night


# Vuelo a eliminar columnas redundantes o de poco valor

In [34]:
columnas_irr2 = ['Day','Hour','resta','city']
fl = fl.drop(columns=columnas_irr2)

In [35]:
mt['Date_time'] = pd.to_datetime(mt['Date_time'])
mt = mt.sort_values('Date_time', ascending = False)
mt.head()

,Metar_id,Date_time,Day,Hour,Condition,Temperature,Wind,Gusts,Relative_hum,Pressure
0,1,2023-10-31 23:30:00,2023-10-31,23:30,Fair,8,3,0,93,1017.0
1,2,2023-10-31 23:00:00,2023-10-31,23:00,Fair,8,1,0,87,1017.0
2,3,2023-10-31 22:30:00,2023-10-31,22:30,Fair,8,0,0,93,1017.0
3,4,2023-10-31 22:00:00,2023-10-31,22:00,Clear,8,0,0,93,1017.0
4,5,2023-10-31 21:30:00,2023-10-31,21:30,Clear,8,0,0,93,1017.0


In [41]:
mt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108890 entries, 0 to 108889
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Metar_id           108890 non-null  int64         
 1   Date_time          108890 non-null  datetime64[ns]
 2   Day                108890 non-null  object        
 3   Hour               108890 non-null  object        
 4   Condition          108890 non-null  object        
 5   Temperature        108890 non-null  int64         
 6   Wind               108890 non-null  int64         
 7   Gusts              108890 non-null  int64         
 8   Relative_hum       108890 non-null  int64         
 9   Pressure           108890 non-null  float64       
 10  Mean_Temperature   108889 non-null  float64       
 11  Mean_Wind          108889 non-null  float64       
 12  Mean_Gusts         108889 non-null  float64       
 13  Mean_Relative_hum  108889 non-null  float64 

# Calculo la media movil de las variables numéricas de los partes Metar según los 10 partes anteriores

In [51]:
# Definir las columnas numéricas y 'Condition' para las cuales calcular medias y modas
columnas_numericas = ['Temperature', 'Wind', 'Gusts', 'Relative_hum', 'Pressure']

# Inicializar las nuevas columnas en el DataFrame mt para las columnas numéricas
for columna in columnas_numericas:
    mt[f'Mean_{columna}'] = mt[columna].iloc[::-1].rolling(window=10, min_periods=1).mean().shift()


mt.fillna(method='backfill', inplace=True)

In [57]:
mt.Mean_Temperature.loc[108889] = mt.Temperature.loc[108889]
mt.Mean_Wind.loc[108889] = mt.Wind.loc[108889]
mt.Mean_Gusts.loc[108889] = mt.Gusts.loc[108889]
mt.Mean_Relative_hum.loc[108889] = mt.Relative_hum.loc[108889]
mt.Mean_Pressure.loc[108889] = mt.Pressure.loc[108889]


C:\Users\daarr\AppData\Local\Temp\ipykernel_145836\933766254.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mt.Mean_Temperature.loc[108889] = mt.Temperature.loc[108889]
C:\Users\daarr\AppData\Local\Temp\ipykernel_145836\933766254.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mt.Mean_Wind.loc[108889] = mt.Wind.loc[108889]
C:\Users\daarr\AppData\Local\Temp\ipykernel_145836\933766254.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

# Calculo la moda movil de la variable 'Condition' de los partes Metar según los 10 partes anteriores

In [92]:
# Función para calcular las dos primeras modas de los 10 registros anteriores
def calcular_modas_anteriores(row, df):
    current_index = row.name
    start_index = current_index + 1
    subset = df.loc[start_index:current_index + 10, 'Condition']
    
    # Calcula las modas utilizando la función mode de scipy.stats
    try:
        moda = mode(subset)[0][0]
        return moda
    except:
        return "-"
        

# Aplica la función a cada fila del DataFrame y almacena los resultados en nuevas columnas
mt['Moda_1'] = mt.apply(calcular_modas_anteriores, df=mt, axis=1)


C:\Users\daarr\AppData\Local\Temp\ipykernel_145836\3479174413.py:9: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  moda = mode(subset)[0][0]
C:\Users\daarr\AppData\Local\Temp\ipykernel_145836\3479174413.py:9: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  moda = mode(subset)[0][0]


In [99]:
mt.Moda_1.loc[108889] = "Fair"

C:\Users\daarr\AppData\Local\Temp\ipykernel_145836\1840201061.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mt.Moda_1.loc[108889] = "Fair"


In [102]:
mt.rename(columns={'Moda_1': 'Mode_Condition'}, inplace=True)

In [103]:
mt.head()

,Metar_id,Date_time,Day,Hour,Condition,Temperature,Wind,Gusts,Relative_hum,Pressure,Mean_Temperature,Mean_Wind,Mean_Gusts,Mean_Relative_hum,Mean_Pressure,Mode_Condition
0,108890,2017-11-01 00:00:00,2017-11-01,00:00,Clear,10,3,0,87,1021.0,10.000000,3.000000,0.0,87.000000,1021.000000,Clear
1,108889,2017-11-01 00:30:00,2017-11-01,00:30,Clear,9,4,0,87,1022.0,10.000000,3.000000,0.0,87.000000,1021.000000,Clear
2,108888,2017-11-01 01:00:00,2017-11-01,01:00,Clear,10,4,0,82,1022.0,9.500000,3.500000,0.0,87.000000,1021.500000,Clear
3,108887,2017-11-01 01:30:00,2017-11-01,01:30,Clear,9,4,0,87,1022.0,9.666667,3.666667,0.0,85.333333,1021.666667,Clear
4,108886,2017-11-01 02:00:00,2017-11-01,02:00,Clear,9,3,0,87,1022.0,9.500000,3.750000,0.0,85.750000,1021.750000,Clear


# Adjudico los valores meteorológicos correspondientes a cada vuelo

In [108]:
fl= fl.sort_values('Date_time')
mt= mt.sort_values('Date_time')

# Realiza el merge asof para encontrar el parte meteorológico más cercano en el tiempo
fl = pd.merge_asof(fl, mt, on='Date_time', direction='backward')


In [110]:
fl.columns

Index(['Date_time', 'cod_flight_IATA', 'week_day', 'cod_airliner_IATA',
       'cod_airport_IATA', 'arr_mins_of_delay', 'duration', 'Metar_id_x',
       'Condition_x', 'Temperature_x', 'Wind_x', 'Gusts_x', 'Relative_hum_x',
       'Pressure_x', 'NumSemana', 'momento_del_dia', 'Metar_id_y', 'Day',
       'Hour', 'Condition_y', 'Temperature_y', 'Wind_y', 'Gusts_y',
       'Relative_hum_y', 'Pressure_y', 'Mean_Temperature', 'Mean_Wind',
       'Mean_Gusts', 'Mean_Relative_hum', 'Mean_Pressure', 'Mode_Condition'],
      dtype='object')

In [115]:
reorder = ['Date_time', 'arr_mins_of_delay', 'week_day','NumSemana', 'momento_del_dia', 'cod_flight_IATA', 'cod_airliner_IATA',
       'cod_airport_IATA', 'duration','Mean_Temperature', 'Mean_Wind',
       'Mean_Gusts', 'Mean_Relative_hum', 'Mean_Pressure', 'Mode_Condition']
fl = fl[reorder]

NameError: name 'Date_time' is not defined

# Renombro algunas columnas

In [114]:
fl.rename(columns={'NumSemana': 'week_num','momento_del_dia':'day_time'}, inplace=True)
fl.tail()

,Date_time,arr_mins_of_delay,week_day,week_num,day_time,cod_flight_IATA,cod_airliner_IATA,cod_airport_IATA,duration,Mean_Temperature,Mean_Wind,Mean_Gusts,Mean_Relative_hum,Mean_Pressure,Mode_Condition
880757,2023-10-31 23:55:00,-35,Tuesday,44,Night,UX41,UX,EZE,12.0,9.2,0.5,0.0,87.3,1016.5,Fair
880758,2023-10-31 23:55:00,-15,Tuesday,44,Night,IB6827,IB,GRU,10.0,9.2,0.5,0.0,87.3,1016.5,Fair
880759,2023-10-31 23:59:00,-6,Tuesday,44,Night,IB6011,IB,MVD,12.0,9.2,0.5,0.0,87.3,1016.5,Fair
880760,2023-10-31 23:59:00,0,Tuesday,44,Night,IB6841,IB,EZE,12.0,9.2,0.5,0.0,87.3,1016.5,Fair
880761,2023-10-31 23:59:00,-6,Tuesday,44,Night,IB6833,IB,SCL,13.0,9.2,0.5,0.0,87.3,1016.5,Fair


In [116]:
fl = fl.drop(columns='Date_time')

In [117]:
fl.to_csv("../data/ML/clean_data_for_ML.csv", index = False)